# 마르코프 체인을 이용한 텍스트 생성

이 노트북에서는 마르코프 체인(Markov Chain)을 구현하여 텍스트를 생성하는 방법을 배워보겠습니다.
마르코프 체인은 현재 상태가 오직 직전 상태에만 의존한다는 가정 하에 작동하는 확률적 모델입니다.
텍스트 생성에서는 다음 단어가 현재 단어에만 의존한다고 가정합니다.


## 1. 필요한 라이브러리 import

마르코프 체인 구현에 필요한 라이브러리들을 import합니다.


In [1]:
import re
import random
from nltk.tokenize import word_tokenize
from collections import defaultdict, deque

print("필요한 라이브러리를 성공적으로 import했습니다.")


필요한 라이브러리를 성공적으로 import했습니다.


## 2. MarkovChain 클래스의 기본 구조

마르코프 체인 클래스를 정의하고 초기화 과정을 살펴보겠습니다.


In [18]:
class MarkovChain:
    def __init__(self):
        """
        마르코프 체인 초기화
        - lookup_dict: 단어 -> 다음 가능한 단어들 목록의 매핑
        - _seeded: 랜덤 시드 설정 여부
        """
        self.lookup_dict = defaultdict(list)
        self._seeded = False
        self.__seed_me()

    def __seed_me(self, rand_seed=None):
        """
        랜덤 시드 설정 메서드
        - rand_seed: 특정 시드값 (None이면 현재 시간 기반으로 자동 설정)
        """
        if self._seeded is not True:
            try:
                if rand_seed is not None:
                    random.seed(rand_seed)
                    print(f"랜덤 시드를 {rand_seed}로 설정했습니다.")
                else:
                    random.seed()
                    print("랜덤 시드를 자동으로 설정했습니다.")
                self._seeded = True
            except NotImplementedError:
                print("랜덤 시드 설정에 실패했습니다.")
                self._seeded = False
    
    def add_document(self, str):
        """
        문서를 마르코프 체인에 추가
        1. 텍스트 전처리
        2. 단어 쌍 생성
        3. lookup_dict에 단어 쌍들 저장
        """
        # 1단계: 전처리
        preprocessed_list = self._preprocess(str)
        
        # 2단계: 단어 쌍 생성
        pairs = self._MarkovChain__generate_tuple_keys(preprocessed_list)
        
        # 3단계: lookup_dict에 저장
        for pair in pairs:
            self.lookup_dict[pair[0]].append(pair[1])
                
    def _preprocess(self, str):
        """
        텍스트 전처리 메서드
        1. 정규표현식으로 특수문자를 공백으로 대체
        2. 소문자로 변환
        3. NLTK word_tokenize로 토큰화
        """
        # 1단계: 특수문자 제거 (알파벳, 숫자가 아닌 문자를 공백으로 대체)
        cleaned = re.sub(r"\\W+", " ", str).lower()
        
        # 2단계: 토큰화
        tokenized = word_tokenize(cleaned)
        
        return tokenized

    def __generate_tuple_keys(self, data):
        """
        연속된 단어 쌍을 생성하는 제너레이터
        [단어1, 단어2, 단어3, 단어4] -> [[단어1, 단어2], [단어2, 단어3], [단어3, 단어4]]
        """
        if len(data) < 1:
            return
        
        for i in range(len(data) - 1):
            yield [data[i], data[i + 1]]

    def generate_text(self, max_length=50):
        """
        마르코프 체인을 사용하여 텍스트 생성
        1. 시작 단어 선택 (첫 번째 키)
        2. 현재 단어에서 가능한 다음 단어들 중 랜덤 선택
        3. max_length에 도달하거나 더 이상 연결할 단어가 없을 때까지 반복
        """
        context = deque()  # 현재 컨텍스트를 저장하는 큐
        output = []        # 생성된 단어들을 저장하는 리스트
        
        if len(self.lookup_dict) > 0:
            # 시드 재설정 (재현 가능한 결과를 위해)
            self._MarkovChain__seed_me(rand_seed=len(self.lookup_dict))
            
            # 시작 단어 선택 (lookup_dict의 첫 번째 키)
            chain_head = [list(self.lookup_dict)[0]]
            context.extend(chain_head)
            
            # max_length-1 만큼 단어 생성 (마지막에 context를 추가하므로)
            while len(output) < (max_length - 1):
                # 현재 단어에서 가능한 다음 단어들 가져오기
                next_choices = self.lookup_dict[context[-1]]
                
                if len(next_choices) > 0:
                    # 가능한 선택지 중에서 랜덤하게 하나 선택
                    next_word = random.choice(next_choices)
                    context.append(next_word)
                    output.append(context.popleft())  # 큐의 앞쪽 원소를 출력에 추가
                else:
                    # 더 이상 연결할 단어가 없으면 종료
                    break
                    
            # 남은 컨텍스트를 출력에 추가
            output.extend(list(context))
            
        return " ".join(output)

# 클래스 인스턴스 생성 테스트
test_chain = MarkovChain()
print(f"초기 lookup_dict 크기: {len(test_chain.lookup_dict)}")


랜덤 시드를 자동으로 설정했습니다.
초기 lookup_dict 크기: 0


## 3. 랜덤 시드 설정 메서드

재현 가능한 결과를 위한 랜덤 시드 설정 메서드를 구현합니다.


In [6]:
# 시드 설정 테스트
print("시드 설정 테스트:")
test_chain = MarkovChain()
print(f"시드 설정 상태: {test_chain._seeded}")

# 특정 시드로 재설정
test_chain._seeded = False
test_chain._MarkovChain__seed_me(42)


시드 설정 테스트:
랜덤 시드를 자동으로 설정했습니다.
MarkovChain 인스턴스가 생성되었습니다.
lookup_dict 타입: <class 'collections.defaultdict'>
시드 설정 완료: True
시드 설정 상태: True
랜덤 시드를 42로 설정했습니다.


## 4. 텍스트 전처리 메서드

입력 텍스트를 정리하고 토큰화하는 전처리 과정을 구현합니다.


In [12]:
# 전처리 테스트
test_chain = MarkovChain()
sample_text = "Hello, World! This is a test. How are you doing today?"

print(f"원본 텍스트: {sample_text}")
processed = test_chain._preprocess(sample_text)
print(f"전처리 결과: {processed}")
print(f"토큰 개수: {len(processed)}")

# 더 복잡한 예시
complex_text = "To be, or not to be--that is the question!"
processed_complex = test_chain._preprocess(complex_text)
print(f"\n복잡한 텍스트: {complex_text}")
print(f"전처리 결과: {processed_complex}")


랜덤 시드를 자동으로 설정했습니다.
MarkovChain 인스턴스가 생성되었습니다.
lookup_dict 타입: <class 'collections.defaultdict'>
시드 설정 완료: True
원본 텍스트: Hello, World! This is a test. How are you doing today?
전처리 결과: ['hello', ',', 'world', '!', 'this', 'is', 'a', 'test', '.', 'how', 'are', 'you', 'doing', 'today', '?']
토큰 개수: 15

복잡한 텍스트: To be, or not to be--that is the question!
전처리 결과: ['to', 'be', ',', 'or', 'not', 'to', 'be', '--', 'that', 'is', 'the', 'question', '!']


## 5. 단어 쌍 생성 메서드

연속된 단어들로부터 (현재단어, 다음단어) 쌍을 생성하는 메서드를 구현합니다.


In [15]:
# 단어 쌍 생성 테스트
test_chain = MarkovChain()
test_words = ["to", "be", "or", "not", "to", "be"]

print(f"입력 단어 리스트: {test_words}")
print("생성된 단어 쌍들:")

pairs = list(test_chain._MarkovChain__generate_tuple_keys(test_words))
for i, pair in enumerate(pairs):
    print(f"  {i+1}: {pair[0]} -> {pair[1]}")

print(f"\n총 {len(pairs)}개의 단어 쌍이 생성되었습니다.")

# 단어 하나만 있는 경우 테스트
single_pairs = list(test_chain._MarkovChain__generate_tuple_keys(["hello"]))
print(f"단어 하나인 경우 생성된 쌍: {len(single_pairs)}개")


랜덤 시드를 자동으로 설정했습니다.
MarkovChain 인스턴스가 생성되었습니다.
lookup_dict 타입: <class 'collections.defaultdict'>
시드 설정 완료: True
입력 단어 리스트: ['to', 'be', 'or', 'not', 'to', 'be']
생성된 단어 쌍들:
  1: to -> be
  2: be -> or
  3: or -> not
  4: not -> to
  5: to -> be

총 5개의 단어 쌍이 생성되었습니다.
단어 하나인 경우 생성된 쌍: 0개


## 6. 문서 추가 메서드

텍스트 문서를 마르코프 체인에 학습시키는 메서드를 구현합니다.


In [ ]:
# 문서 추가 테스트
test_chain = MarkovChain()

# 간단한 테스트 문서
test_doc1 = "The cat sat on the mat. The mat was comfortable."
print(f"추가할 문서 1: {test_doc1}")

test_chain.add_document(test_doc1)
print(f"문서 추가 후 lookup_dict 크기: {len(test_chain.lookup_dict)}")

# lookup_dict 내용 확인
print("\nlookup_dict 내용:")
for key, values in test_chain.lookup_dict.items():
    print(f"  '{key}' -> {values}")

# 두 번째 문서 추가
test_doc2 = "The cat likes the mat. The cat sat there."
print(f"\n추가할 문서 2: {test_doc2}")

test_chain.add_document(test_doc2)
print(f"문서 추가 후 lookup_dict 크기: {len(test_chain.lookup_dict)}")

# 업데이트된 lookup_dict 내용 확인
print("\n업데이트된 lookup_dict 내용:")
for key, values in test_chain.lookup_dict.items():
    print(f"  '{key}' -> {values}")


랜덤 시드를 자동으로 설정했습니다.
MarkovChain 인스턴스가 생성되었습니다.
lookup_dict 타입: <class 'collections.defaultdict'>
시드 설정 완료: True
추가할 문서 1: The cat sat on the mat. The mat was comfortable.
문서 추가 후 lookup_dict 크기: 8
\nlookup_dict 내용:
  'the' -> ['cat', 'mat', 'mat']
  'cat' -> ['sat']
  'sat' -> ['on']
  'on' -> ['the']
  'mat' -> ['.', 'was']
  '.' -> ['the']
  'was' -> ['comfortable']
  'comfortable' -> ['.']
\n추가할 문서 2: The cat likes the mat. The cat sat there.
문서 추가 후 lookup_dict 크기: 10
\n업데이트된 lookup_dict 내용:
  'the' -> ['cat', 'mat', 'mat', 'cat', 'mat', 'cat']
  'cat' -> ['sat', 'likes', 'sat']
  'sat' -> ['on', 'there']
  'on' -> ['the']
  'mat' -> ['.', 'was', '.']
  '.' -> ['the', 'the']
  'was' -> ['comfortable']
  'comfortable' -> ['.']
  'likes' -> ['the']
  'there' -> ['.']


## 7. 텍스트 생성 메서드

학습된 마르코프 체인을 사용하여 새로운 텍스트를 생성하는 메서드를 구현합니다.


In [20]:
# 텍스트 생성 테스트
print("텍스트 생성 테스트:")
test_chain = MarkovChain()

# 간단한 훈련 데이터
training_text = """
The cat sat on the mat. The cat was happy. The mat was soft.
The dog ran in the park. The dog was excited. The park was big.
The cat and the dog played together. They were happy friends.
"""

test_chain.add_document(training_text)
print(f"훈련 후 lookup_dict 크기: {len(test_chain.lookup_dict)}")

# 다양한 길이로 텍스트 생성
for length in [10, 20, 30]:
    generated = test_chain.generate_text(max_length=length)
    print(f"\n길이 {length}로 생성된 텍스트:")
    print(f"  {generated}")
    print(f"  (실제 단어 수: {len(generated.split())})")


텍스트 생성 테스트:
랜덤 시드를 자동으로 설정했습니다.
훈련 후 lookup_dict 크기: 21

길이 10로 생성된 텍스트:
  the cat sat on the dog ran in the mat
  (실제 단어 수: 10)

길이 20로 생성된 텍스트:
  the park was happy friends . the park . the mat . the park was soft . the mat was
  (실제 단어 수: 20)

길이 30로 생성된 텍스트:
  the cat sat on the dog ran in the mat . the cat and the dog played together . the dog was excited . they were happy . they were
  (실제 단어 수: 30)


## 8. 햄릿 텍스트로 실제 테스트

실제 햄릿 텍스트를 사용하여 마르코프 체인을 훈련하고 텍스트를 생성해봅시다.


In [21]:
# 햄릿 텍스트 로드 및 마르코프 체인 훈련
try:
    with open("../../data/hamlet.txt", "r", encoding="utf-8") as f:
        hamlet_text = f.read()
    print("햄릿 텍스트를 성공적으로 로드했습니다.")
    print(f"텍스트 길이: {len(hamlet_text):,} 문자")
    print(f"처음 200자: {hamlet_text[:200]}...")
    
except FileNotFoundError:
    print("햄릿 파일을 찾을 수 없습니다. 샘플 텍스트를 사용합니다.")
    hamlet_text = """
    To be or not to be, that is the question. Whether tis nobler in the mind to suffer
    the slings and arrows of outrageous fortune, or to take arms against a sea of troubles
    and by opposing end them. To die, to sleep, no more, and by a sleep to say we end
    the heartache and the thousand natural shocks that flesh is heir to.
    """

# 마르코프 체인 생성 및 훈련
HMM = MarkovChain()
HMM.add_document(hamlet_text)

print(f"\n훈련 완료!")
print(f"학습된 단어 수: {len(HMM.lookup_dict)}")

# 일부 단어의 다음 단어 후보들 확인
sample_words = ['the', 'to', 'and', 'be', 'of']
print("\n일부 단어들의 다음 단어 후보들:")
for word in sample_words:
    if word in HMM.lookup_dict:
        next_words = HMM.lookup_dict[word]
        print(f"  '{word}' -> {len(next_words)}개 선택지: {next_words[:10]}{'...' if len(next_words) > 10 else ''}")
    else:
        print(f"  '{word}' -> 단어를 찾을 수 없습니다.")


햄릿 텍스트를 성공적으로 로드했습니다.
텍스트 길이: 221,777 문자
처음 200자: The Project Gutenberg EBook of Hamlet, by William Shakespeare

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-...
랜덤 시드를 자동으로 설정했습니다.

훈련 완료!
학습된 단어 수: 6785

일부 단어들의 다음 단어 후보들:
  'the' -> 1460개 선택지: ['project', 'use', 'terms', 'project', 'online', 'book', 'bottom', 'footnotes', 'end', 'word']...
  'to' -> 822개 선택지: ['unmix', 'act', 'act', 'the', 'the', 'hamlet_', 'polonius_', 'command', 'future', 'be']...
  'and' -> 808개 선택지: ['with', 'the', 'evans', 'evans', 'admiration', 'as', 'amiable', 'the', 'most', 'feeling']...
  'be' -> 214개 선택지: ['on', 'found', 'easily', 'selected', 'spoke', 'any', 'done', 'green', 'contracted', 'thine']...
  'of' -> 913개 선택지: ['hamlet', 'anyone', 'the', 'this', '_hamlet_', 'each', 'each', 'hamlet', 'denmark', 'denmark_']...


## 10. 다양한 길이의 텍스트 생성

훈련된 마르코프 체인으로 다양한 길이의 텍스트를 생성해봅시다.


In [23]:
# 다양한 길이의 텍스트 생성
lengths = [15, 25, 35, 50]

print("햄릿 스타일 텍스트 생성 결과:")
print("=" * 60)

for i, length in enumerate(lengths, 1):
    generated = HMM.generate_text(max_length=length)
    actual_length = len(generated.split())
    
    print(f"\n{i}. 목표 길이: {length}단어 (실제: {actual_length}단어)")
    print(f"   {generated}")

# 여러 번 생성하여 다양성 확인
print("\n" + "=" * 60)
print("25단어 길이로 5번 생성 (다양성 확인):")
print("-" * 40)

for i in range(5):
    # 시드를 다시 설정하여 다른 결과 생성
    HMM._seeded = False
    generated = HMM.generate_text(max_length=25)
    print(f"{i+1}. {generated}")


햄릿 스타일 텍스트 생성 결과:

1. 목표 길이: 15단어 (실제: 15단어)
   the hilts , [ footnote iv.16 : _green_ ; you to her ? _fran._ for

2. 목표 길이: 25단어 (실제: 25단어)
   the sixteenth century afterwards . polonius , or reason and passion will not beteem_ ] the loss your heart , they say ) _king._ (

3. 목표 길이: 35단어 (실제: 35단어)
   the important foe . _dan._ we may colour [ footnote iii.91 : [ 44 ] contumacious towards . _pol._ ( r. ) that rots itself more of't ; the danger of in his wit ,

4. 목표 길이: 50단어 (실제: 50단어)
   the owner of playing upon me from the time , and the terms of my beard , he was once in this folly drowns it._ ] _laer._ ( r. ) custom which the modesty as mad : _this grave rain many others . ] _queen._ ( c. centre of our

25단어 길이로 5번 생성 (다양성 확인):
----------------------------------------
랜덤 시드를 6785로 설정했습니다.
1. the carpenter ? _ ] freeze thy mother 's daughter._ ] [ 48 ] ah , rosencrantz . [ 93 ] _i.e._ , folded .
랜덤 시드를 6785로 설정했습니다.
2. the carpenter ? _ ] freeze thy mother 's daughter._ ] [ 48 ] a

## 11. 마르코프 체인 분석

학습된 마르코프 체인의 특성을 분석해보겠습니다.


In [24]:
# 마르코프 체인 통계 분석
print("마르코프 체인 분석 결과:")
print("=" * 50)

# 전체 통계
total_words = len(HMM.lookup_dict)
total_transitions = sum(len(transitions) for transitions in HMM.lookup_dict.values())

print(f"고유 단어 수: {total_words:,}")
print(f"전체 전이(transition) 수: {total_transitions:,}")
print(f"평균 전이 수 per 단어: {total_transitions/total_words:.2f}")

# 가장 많은 다음 단어 선택지를 가진 단어들
word_choice_counts = [(word, len(choices)) for word, choices in HMM.lookup_dict.items()]
word_choice_counts.sort(key=lambda x: x[1], reverse=True)

print("\n가장 많은 선택지를 가진 단어들 (Top 10):")
for i, (word, count) in enumerate(word_choice_counts[:10], 1):
    print(f"  {i:2d}. '{word}': {count}개 선택지")

# 가장 적은 선택지를 가진 단어들
print("\n가장 적은 선택지를 가진 단어들 (Bottom 10):")
for i, (word, count) in enumerate(word_choice_counts[-10:], 1):
    print(f"  {i:2d}. '{word}': {count}개 선택지")

# 선택지 수의 분포
choice_counts = [count for _, count in word_choice_counts]
print(f"\n선택지 수 통계:")
print(f"  최대: {max(choice_counts)}")
print(f"  최소: {min(choice_counts)}")
print(f"  평균: {sum(choice_counts)/len(choice_counts):.2f}")
print(f"  중간값: {sorted(choice_counts)[len(choice_counts)//2]}")

# 특정 단어들의 전이 확인
print("\n특정 단어들의 다음 단어 분석:")
interesting_words = ['to', 'the', 'be', 'that', 'and']
for word in interesting_words:
    if word in HMM.lookup_dict:
        choices = HMM.lookup_dict[word]
        # 각 선택지의 빈도 계산
        from collections import Counter
        choice_freq = Counter(choices)
        top_choices = choice_freq.most_common(5)
        print(f"  '{word}' -> 총 {len(choices)}개 전이:")
        for next_word, freq in top_choices:
            percentage = (freq/len(choices))*100
            print(f"    '{next_word}': {freq}회 ({percentage:.1f}%)")


마르코프 체인 분석 결과:
고유 단어 수: 6,785
전체 전이(transition) 수: 47,019
평균 전이 수 per 단어: 6.93

가장 많은 선택지를 가진 단어들 (Top 10):
   1. ',': 3442개 선택지
   2. '.': 1696개 선택지
   3. ']': 1600개 선택지
   4. 'the': 1460개 선택지
   5. '[': 1131개 선택지
   6. 'of': 913개 선택지
   7. ':': 882개 선택지
   8. 'to': 822개 선택지
   9. 'and': 808개 선택지
  10. 'a': 661개 선택지

가장 적은 선택지를 가진 단어들 (Bottom 10):
   1. 'confirmed': 1개 선택지
   2. 'necessarily': 1개 선택지
   3. 'main': 1개 선택지
   4. 'pg': 1개 선택지
   5. 'facility': 1개 선택지
   6. '//www.gutenberg.org': 1개 선택지
   7. 'includes': 1개 선택지
   8. 'produce': 1개 선택지
   9. 'subscribe': 1개 선택지
  10. 'newsletter': 1개 선택지

선택지 수 통계:
  최대: 3442
  최소: 1
  평균: 6.93
  중간값: 1

특정 단어들의 다음 단어 분석:
  'to' -> 총 822개 전이:
    'the': 89회 (10.8%)
    'be': 45회 (5.5%)
    'you': 22회 (2.7%)
    'a': 19회 (2.3%)
    'his': 18회 (2.2%)
  'the' -> 총 1460개 전이:
    'king': 34회 (2.3%)
    'project': 31회 (2.1%)
    'same': 16회 (1.1%)
    'very': 15회 (1.0%)
    'world': 13회 (0.9%)
  'be' -> 총 214개 전이:
    ',': 13회 (6.1%)
    'the': 

## 12. 마르코프 체인의 한계와 개선점

마르코프 체인의 장단점을 살펴보고, 실제 생성 결과를 분석해봅시다.


In [25]:
# 마르코프 체인의 특성 분석

print("마르코프 체인의 장단점 분석:")
print("=" * 50)

print("\n✅ 장점:")
print("  1. 구현이 간단하고 직관적")
print("  2. 학습 속도가 빠름")
print("  3. 메모리 효율적")
print("  4. 원본 텍스트의 국부적 패턴을 잘 학습")

print("\n❌ 단점:")
print("  1. 장거리 의존성(long-range dependency)를 포착할 수 없음")
print("  2. 문법적 일관성이 떨어질 수 있음")
print("  3. 의미적 연관성이 부족할 수 있음")
print("  4. 1차 마르코프 체인은 너무 단순함")

# 개선 방안 시뮬레이션: 2차 마르코프 체인 아이디어
print("\n🔧 개선 방안:")
print("  1. 고차 마르코프 체인 (2-gram, 3-gram 등)")
print("  2. 평활화(Smoothing) 기법 적용")
print("  3. 백오프(Backoff) 전략")
print("  4. 가중치 기반 선택")

# 실제 생성 텍스트의 문제점 분석
print("\n📊 생성된 텍스트 품질 분석:")
sample_generations = []
for i in range(5):
    HMM._seeded = False
    text = HMM.generate_text(max_length=20)
    sample_generations.append(text)
    print(f"  {i+1}. {text}")

print("\n분석 결과:")
print("  - 문법적 완성도: 중간 수준")
print("  - 의미적 일관성: 낮음")
print("  - 원본 텍스트와의 유사성: 높음")
print("  - 창의성: 제한적")

# 원본 텍스트와의 비교
print("\n원본 햄릿에서 실제 문장 예시:")
if len(hamlet_text) > 1000:
    # 원본에서 몇 개 문장 추출
    sentences = hamlet_text.split('.')[:3]
    for i, sentence in enumerate(sentences, 1):
        clean_sentence = ' '.join(sentence.split())[:80]
        if clean_sentence:
            print(f"  {i}. {clean_sentence}...")


마르코프 체인의 장단점 분석:

✅ 장점:
  1. 구현이 간단하고 직관적
  2. 학습 속도가 빠름
  3. 메모리 효율적
  4. 원본 텍스트의 국부적 패턴을 잘 학습

❌ 단점:
  1. 장거리 의존성(long-range dependency)를 포착할 수 없음
  2. 문법적 일관성이 떨어질 수 있음
  3. 의미적 연관성이 부족할 수 있음
  4. 1차 마르코프 체인은 너무 단순함

🔧 개선 방안:
  1. 고차 마르코프 체인 (2-gram, 3-gram 등)
  2. 평활화(Smoothing) 기법 적용
  3. 백오프(Backoff) 전략
  4. 가중치 기반 선택

📊 생성된 텍스트 품질 분석:
랜덤 시드를 6785로 설정했습니다.
  1. the carpenter ? _ ] freeze thy mother 's daughter._ ] [ 48 ] ah , rosencrantz . [ 93
랜덤 시드를 6785로 설정했습니다.
  2. the carpenter ? _ ] freeze thy mother 's daughter._ ] [ 48 ] ah , rosencrantz . [ 93
랜덤 시드를 6785로 설정했습니다.
  3. the carpenter ? _ ] freeze thy mother 's daughter._ ] [ 48 ] ah , rosencrantz . [ 93
랜덤 시드를 6785로 설정했습니다.
  4. the carpenter ? _ ] freeze thy mother 's daughter._ ] [ 48 ] ah , rosencrantz . [ 93
랜덤 시드를 6785로 설정했습니다.
  5. the carpenter ? _ ] freeze thy mother 's daughter._ ] [ 48 ] ah , rosencrantz . [ 93

분석 결과:
  - 문법적 완성도: 중간 수준
  - 의미적 일관성: 낮음
  - 원본 텍스트와의 유사성: 높음
  - 창의성: 제한적

원본 햄릿에서 실제 문장 예시:
  1. 

## 결론

이 노트북에서 우리는:

1. **마르코프 체인의 기본 개념**을 이해했습니다
2. **텍스트 전처리와 단어 쌍 생성** 과정을 학습했습니다  
3. **확률적 텍스트 생성** 방법을 구현했습니다
4. **실제 문학 작품을 활용한 텍스트 생성**을 수행했습니다
5. **마르코프 체인의 특성과 한계**를 분석했습니다

마르코프 체인은 자연어 처리와 텍스트 생성의 기초적이면서도 중요한 방법입니다. 비록 현대의 트랜스포머 모델들에 비해 단순하지만, 확률적 언어 모델의 기본 개념을 이해하는 데 매우 유용합니다.

### 주요 학습 포인트:
- **마르코프 가정**: 현재 상태는 직전 상태에만 의존
- **전이 확률**: 단어 간의 연결 빈도를 통한 확률 계산
- **랜덤 생성**: 확률 분포에 따른 다음 단어 선택
- **국부적 일관성**: 인접한 단어들 간의 자연스러운 연결

마르코프 체인을 통해 배운 이러한 개념들은 더 복잡한 언어 모델을 이해하는 기초가 됩니다!
